In [5]:
import os

import pandas as pd
import parse
import numpy as np
import multihash
import rdflib as rl
import urllib
#import prov.model as pm

bucket_url_prefix = 'https://s3.amazonaws.com/fcp-indi/'
info_file = 'fcp-indi'

In [2]:
with open(info_file, 'rt') as fp:
    projects = []
    for val in fp.readlines():
        if val.strip():
            projects.append(val.strip().split()[4].split('/')[2])
np.unique(projects)

array(['ABIDE_BIDS', 'ABIDE_Initiative', 'ACPI', 'ADHD200',
       'BrainGenomicsSuperstructProject', 'CC_ME', 'CORR',
       'CPAC_Regression_Test', 'HBNSS', 'HBNSSI', 'INDI', 'RocklandSample'], 
      dtype='<U31')

In [3]:
with open(info_file, 'rt') as fp:
    pheno = [val.strip().split()[-1] for val in fp.readlines() if 
             any([key in val.lower() for key in ['participants.tsv', 'pheno']])
             and all([key not in val.lower() for key in ['outputs', 'tsv.gz', 'cpac_regression_test', 
                                                         'events.tsv', 'sessions.tsv', 'tar.gz']])]
pheno

['data/Projects/ABIDE_Initiative/PhenotypicData/phenotypic_CALTECH.csv',
 'data/Projects/ABIDE_Initiative/PhenotypicData/phenotypic_CMU.csv',
 'data/Projects/ABIDE_Initiative/PhenotypicData/phenotypic_KKI.csv',
 'data/Projects/ABIDE_Initiative/PhenotypicData/phenotypic_LEUVEN_1.csv',
 'data/Projects/ABIDE_Initiative/PhenotypicData/phenotypic_LEUVEN_2.csv',
 'data/Projects/ABIDE_Initiative/PhenotypicData/phenotypic_MAX_MUN.csv',
 'data/Projects/ABIDE_Initiative/PhenotypicData/phenotypic_NYU.csv',
 'data/Projects/ABIDE_Initiative/PhenotypicData/phenotypic_OHSU.csv',
 'data/Projects/ABIDE_Initiative/PhenotypicData/phenotypic_OLIN.csv',
 'data/Projects/ABIDE_Initiative/PhenotypicData/phenotypic_PITT.csv',
 'data/Projects/ABIDE_Initiative/PhenotypicData/phenotypic_SBL.csv',
 'data/Projects/ABIDE_Initiative/PhenotypicData/phenotypic_SDSU.csv',
 'data/Projects/ABIDE_Initiative/PhenotypicData/phenotypic_STANFORD.csv',
 'data/Projects/ABIDE_Initiative/PhenotypicData/phenotypic_TRINITY(1).csv',


In [6]:
print(len(pheno))
if not os.path.exists('pheno'):
    os.mkdir('pheno')
localpheno = []
for path in pheno:
    url = bucket_url_prefix + path
    parsed = parse.parse('data/Projects/{project}/{frag}', path).named
    filename = os.path.abspath(os.path.join('pheno', parsed['project'] + '-' + path.split('/')[-1]))
    parsed['url'] = url
    parsed['filename'] = filename
    if not os.path.exists(filename):
        try:
            urllib.request.urlretrieve(url, filename)
            localpheno.append(parsed)
        except urllib.error.HTTPError as e:
            print(url)
    else:
        localpheno.append(parsed)
localpheno

62


[{'filename': '/om/user/satra/projects/metasearch/crawler/pheno/ABIDE_Initiative-phenotypic_CALTECH.csv',
  'frag': 'PhenotypicData/phenotypic_CALTECH.csv',
  'project': 'ABIDE_Initiative',
  'url': 'https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/PhenotypicData/phenotypic_CALTECH.csv'},
 {'filename': '/om/user/satra/projects/metasearch/crawler/pheno/ABIDE_Initiative-phenotypic_CMU.csv',
  'frag': 'PhenotypicData/phenotypic_CMU.csv',
  'project': 'ABIDE_Initiative',
  'url': 'https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/PhenotypicData/phenotypic_CMU.csv'},
 {'filename': '/om/user/satra/projects/metasearch/crawler/pheno/ABIDE_Initiative-phenotypic_KKI.csv',
  'frag': 'PhenotypicData/phenotypic_KKI.csv',
  'project': 'ABIDE_Initiative',
  'url': 'https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/PhenotypicData/phenotypic_KKI.csv'},
 {'filename': '/om/user/satra/projects/metasearch/crawler/pheno/ABIDE_Initiative-phenotypic_LEUVEN_1.csv',

In [7]:
with open(info_file, 'rt') as fp:
    anat = [val.strip().split()[-1] for val in fp.readlines() if 
            any([key in val.lower() for key in ['anat', 't1', 'mprage']])
            and all([key not in val.lower() for key in ['tsv.gz', 'cpac_regression_test', 
                                                        'cc_me', 'average', '.ds_store',
                                                        'tar.gz', 'json', 'bold']])]
anat = [val for val in anat if 't1.mgz' in val.lower() or 'output' not in val.lower()]

In [8]:
len(anat)

10666

In [9]:
patterns = ['data/Projects/{study}/RawData/{site}/{subj_id}/session_{session_id}/anat_{anat_id}/mprage.nii.gz',
            'data/Projects/{study}/Outputs/freesurfer/5.1/{site}_{site_suffix}_{subj_id}/mri/T1.mgz',
            'data/Projects/{study}/Outputs/freesurfer/5.1/{site}_{subj_id}/mri/T1.mgz',
            'data/Projects/{study}/surfaces/freesurfer/5.3/{subj_id}/mri/T1.mgz',
            'data/Projects/{study}/Outputs/IBA_TRT/freesurfer_gpu/{subj_id}-session_{session_id}/mri/T1.mgz',
            'data/Projects/{study}/Outputs/IBA_TRT/freesurfer/{subj_id}-session_{session_id}/mri/T1.mgz',
            'data/Projects/{study}/RawData/{site}/{subj_id}/session_{session_id}/anat_{anat_id}/anat.nii.gz',
            'data/Projects/{study:w}/sub-{subj_id}/ses-{session_id}/anat/sub-{subj_id}_ses-{session_id}_T1w.nii.gz',
            'data/Projects/{study:w}/RawData/{subj_id}/{session_id}/{deface_id}/{deface_id2}_defaced.nii.gz',
            'data/Projects/INDI/{study}/RawData/sub-{subj_id}/anat/sub-{subj_id}_T1w.nii.gz',
           ]

info = []
remainder = []
for val in anat:
    got_match = False
    for pattern in patterns:
        parsed = parse.parse(pattern, val)
        if parsed:
            parsed.named['atLocation'] = bucket_url_prefix + val
            parsed.named['pattern'] = pattern
            info.append(parsed.named)
            got_match = True
            break
    if not got_match:
        remainder.append(val)
remainder

['data/Projects/ABIDE_BIDS/CMU_a/sub-0050642/anat/sub-0050642_T1w.nii.gz',
 'data/Projects/ABIDE_BIDS/CMU_a/sub-0050646/anat/sub-0050646_T1w.nii.gz',
 'data/Projects/ABIDE_BIDS/CMU_a/sub-0050647/anat/sub-0050647_T1w.nii.gz',
 'data/Projects/ABIDE_BIDS/CMU_a/sub-0050649/anat/sub-0050649_T1w.nii.gz',
 'data/Projects/ABIDE_BIDS/CMU_a/sub-0050653/anat/sub-0050653_T1w.nii.gz',
 'data/Projects/ABIDE_BIDS/CMU_a/sub-0050654/anat/sub-0050654_T1w.nii.gz',
 'data/Projects/ABIDE_BIDS/CMU_a/sub-0050656/anat/sub-0050656_T1w.nii.gz',
 'data/Projects/ABIDE_BIDS/CMU_a/sub-0050659/anat/sub-0050659_T1w.nii.gz',
 'data/Projects/ABIDE_BIDS/CMU_a/sub-0050660/anat/sub-0050660_T1w.nii.gz',
 'data/Projects/ABIDE_BIDS/CMU_a/sub-0050663/anat/sub-0050663_T1w.nii.gz',
 'data/Projects/ABIDE_BIDS/CMU_a/sub-0050664/anat/sub-0050664_T1w.nii.gz',
 'data/Projects/ABIDE_BIDS/CMU_a/sub-0050665/anat/sub-0050665_T1w.nii.gz',
 'data/Projects/ABIDE_BIDS/CMU_a/sub-0050666/anat/sub-0050666_T1w.nii.gz',
 'data/Projects/ABIDE_BID

In [10]:
len(remainder), len(info)

(1081, 9585)

In [11]:
url = 'https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/RawData/CMU_a/0050642/session_1/anat_1/mprage.nii.gz'
response = urllib.request.urlopen(url)
filedata = response.read()

In [12]:
encoded = multihash.encode(filedata, multihash.SHA2_256)
print(''.join(format(x, '02x') for x in encoded))

1220d03b3d865c212890578d06c1242406813aae11b9fb8e201b55c282b94bf33687


In [13]:
localpheno[:3]

[{'filename': '/om/user/satra/projects/metasearch/crawler/pheno/ABIDE_Initiative-phenotypic_CALTECH.csv',
  'frag': 'PhenotypicData/phenotypic_CALTECH.csv',
  'project': 'ABIDE_Initiative',
  'url': 'https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/PhenotypicData/phenotypic_CALTECH.csv'},
 {'filename': '/om/user/satra/projects/metasearch/crawler/pheno/ABIDE_Initiative-phenotypic_CMU.csv',
  'frag': 'PhenotypicData/phenotypic_CMU.csv',
  'project': 'ABIDE_Initiative',
  'url': 'https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/PhenotypicData/phenotypic_CMU.csv'},
 {'filename': '/om/user/satra/projects/metasearch/crawler/pheno/ABIDE_Initiative-phenotypic_KKI.csv',
  'frag': 'PhenotypicData/phenotypic_KKI.csv',
  'project': 'ABIDE_Initiative',
  'url': 'https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/PhenotypicData/phenotypic_KKI.csv'}]

In [14]:
df = pd.read_csv(localpheno[0]['filename'])

In [15]:
df.DX_GROUP.unique()

array([1, 2])

In [16]:
info_df = pd.DataFrame(info)

In [17]:
info_df.study.unique()

array(['ABIDE_Initiative', 'ACPI', 'ADHD200',
       'BrainGenomicsSuperstructProject', 'CORR', 'HypnosisBarrios',
       'RocklandSample'], dtype=object)

In [18]:
info_df.head()

,anat_id,atLocation,deface_id,deface_id2,pattern,session_id,site,site_suffix,study,subj_id
0,NaN,https://s3.amazonaws.com/fcp-indi/data/Project...,NaN,NaN,data/Projects/{study}/Outputs/freesurfer/5.1/{...,NaN,CMU,a,ABIDE_Initiative,0050642
1,NaN,https://s3.amazonaws.com/fcp-indi/data/Project...,NaN,NaN,data/Projects/{study}/Outputs/freesurfer/5.1/{...,NaN,CMU,a,ABIDE_Initiative,0050646
2,NaN,https://s3.amazonaws.com/fcp-indi/data/Project...,NaN,NaN,data/Projects/{study}/Outputs/freesurfer/5.1/{...,NaN,CMU,a,ABIDE_Initiative,0050647
3,NaN,https://s3.amazonaws.com/fcp-indi/data/Project...,NaN,NaN,data/Projects/{study}/Outputs/freesurfer/5.1/{...,NaN,CMU,a,ABIDE_Initiative,0050649
4,NaN,https://s3.amazonaws.com/fcp-indi/data/Project...,NaN,NaN,data/Projects/{study}/Outputs/freesurfer/5.1/{...,NaN,CMU,a,ABIDE_Initiative,0050653


In [19]:
abide_mri_df = info_df[info_df.study == 'ABIDE_Initiative']

In [20]:
abide_mri_df.subj_id.unique().shape == abide_mri_df.subj_id.shape

False

In [21]:
url_dfs = []
for val in info_df.study.unique():
    print(val, info_df[info_df.study == val].shape, info_df[info_df.study == val].subj_id.unique().shape)
    study_df = info_df[info_df.study == val]
    foo = study_df[['atLocation', 'subj_id']] #, 'session_id']]
    '''
    try:
        subj_id_new = foo.subj_id.apply(lambda x: val.lower() + '_%d' % int(x) )
    except ValueError:
        subj_id_new = foo.subj_id.apply(lambda x: val.lower() + '_' + x)
    foo.loc[:, 'subj_id'] = subj_id_new
    foo.index = subj_id_new.values
    '''
    foo.to_csv(val + '.csv', header=False, index=False, )
    url_dfs.append(foo)

ABIDE_Initiative (2214, 10) (1112,)
ACPI (129, 10) (129,)
ADHD200 (1927, 10) (973,)
BrainGenomicsSuperstructProject (1570, 10) (1570,)
CORR (3203, 10) (1520,)
HypnosisBarrios (10, 10) (10,)
RocklandSample (532, 10) (421,)


In [150]:
from joblib import Parallel, delayed
import multiprocessing

def applyParallel(df, func):
    retLst = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(func)(url) for url in pd.DataFrame(df).iterrows())
    return pd.concat(retLst)

def compute_hash(url):
    csv_file = os.path.join('hashfiles', url[1].atLocation.replace('https://', '') + '.csv')
    if os.path.exists(csv_file):
        return pd.read_csv(csv_file)
    try:
        response = urllib.request.urlopen(url[1].atLocation)
        filedata = response.read()
    except urllib.error.HTTPError:
        return pd.DataFrame({'multihash': [pd.np.nan]}, index=[url[0]])
    encoded = multihash.encode(filedata, multihash.SHA2_256)
    df = pd.DataFrame({'multihash': [''.join(format(x, '02x') for x in encoded)]}, index=[url[0]])
    df.to_csv(csv_file)
    return df

In [142]:
val = list(pd.DataFrame(info_df['atLocation'][:2]).iterrows())[0]
val[1].atLocation

pd.DataFrame({'multihash': [''.join(format(x, 's') for x in val[1].atLocation)]}, index=[1])

,multihash
1,https://s3.amazonaws.com/fcp-indi-new/data/Pro...


hash_df = applyParallel(info_df['atLocation'], compute_hash)
hash_df.head()

In [280]:
project_pheno = ['data/Projects/ABIDE_Initiative/Phenotypic_V1_0b.csv',
 'data/Projects/ACPI/PhenotypicData/acpi_aggregated_phenotypic_data.csv',
 'data/Projects/ADHD200/RawData/Brown/Brown_TestRelease_phenotypic.csv',
 'data/Projects/ADHD200/RawData/Brown_TestRelease_phenotypic.csv',
 'data/Projects/ADHD200/RawData/KKI/KKI_phenotypic.csv',
 'data/Projects/ADHD200/RawData/KKI_phenotypic.csv',
 'data/Projects/ADHD200/RawData/NYU/NYU_phenotypic.csv',
 'data/Projects/ADHD200/RawData/NYU_phenotypic.csv',
 'data/Projects/ADHD200/RawData/NeuroIMAGE/NeuroIMAGE_TestRelease_phenotypic.csv',
 'data/Projects/ADHD200/RawData/NeuroIMAGE/NeuroIMAGE_phenotypic.csv',
 'data/Projects/ADHD200/RawData/NeuroIMAGE_phenotypic.csv',
 'data/Projects/ADHD200/RawData/OHSU/OHSU_TestRelease_phenotypic.csv',
 'data/Projects/ADHD200/RawData/OHSU_TestRelease_phenotypic.csv',
 'data/Projects/ADHD200/RawData/OHSU_phenotypic.csv',
 'data/Projects/ADHD200/RawData/Peking_1/Peking_1_phenotypic.csv',
 'data/Projects/ADHD200/RawData/Peking_1_TestRelease_phenotypic.csv',
 'data/Projects/ADHD200/RawData/Peking_1_phenotypic.csv',
 'data/Projects/ADHD200/RawData/Pittsburgh/Pittsburgh_phenotypic.csv',
 'data/Projects/ADHD200/RawData/Pittsburgh_phenotypic.csv',
 'data/Projects/ADHD200/RawData/WashU/WashU_phenotypic.csv',
 'data/Projects/CORR/RawData/CoRR_AggregatedPhenotypicData.csv',
 'data/Projects/INDI/HypnosisBarrios/RawData/participants.tsv',
 'data/Projects/RocklandSample/PhenotypicData/nki-rs_lite_r6_phenotypic_v1.csv',
 'data/Projects/RocklandSample/PhenotypicData/nki-rs_lite_r7_phenotypic_v1.csv',
 'data/Projects/RocklandSample/PhenotypicData/nki-rs_lite_r8_phenotypic_v1.csv',
 'data/Projects/RocklandSample/RawDataBIDS/participants.tsv']

In [281]:
csv2use = [val2 for val2 in localpheno for val in project_pheno if val in val2['url']]

In [282]:
csv2use[0]

{'filename': '/software/data/brainbox/pheno/ABIDE_Initiative-Phenotypic_V1_0b.csv',
 'frag': 'Phenotypic_V1_0b.csv',
 'project': 'ABIDE_Initiative',
 'url': 'https://s3.amazonaws.com/fcp-indi-new/data/Projects/ABIDE_Initiative/Phenotypic_V1_0b.csv'}

In [283]:
data_dfs = []
for val in csv2use:
    print(val['filename'])
    if 'csv' in val['filename']:
        df = pd.read_csv(val['filename'], na_values=['-9999'], dtype=object)
    else:
        df = pd.read_table(val['filename'], na_values=['-9999'], dtype=object)
    df['Project'] = val['project']
    data_dfs.append(df)

/software/data/brainbox/pheno/ABIDE_Initiative-Phenotypic_V1_0b.csv
/software/data/brainbox/pheno/ACPI-acpi_aggregated_phenotypic_data.csv
/software/data/brainbox/pheno/ADHD200-Brown_TestRelease_phenotypic.csv
/software/data/brainbox/pheno/ADHD200-Brown_TestRelease_phenotypic.csv
/software/data/brainbox/pheno/ADHD200-KKI_phenotypic.csv
/software/data/brainbox/pheno/ADHD200-KKI_phenotypic.csv
/software/data/brainbox/pheno/ADHD200-NYU_phenotypic.csv
/software/data/brainbox/pheno/ADHD200-NYU_phenotypic.csv
/software/data/brainbox/pheno/ADHD200-NeuroIMAGE_TestRelease_phenotypic.csv
/software/data/brainbox/pheno/ADHD200-NeuroIMAGE_phenotypic.csv
/software/data/brainbox/pheno/ADHD200-NeuroIMAGE_phenotypic.csv
/software/data/brainbox/pheno/ADHD200-OHSU_TestRelease_phenotypic.csv
/software/data/brainbox/pheno/ADHD200-OHSU_TestRelease_phenotypic.csv
/software/data/brainbox/pheno/ADHD200-OHSU_phenotypic.csv
/software/data/brainbox/pheno/ADHD200-Peking_1_phenotypic.csv
/software/data/brainbox/phe

In [284]:
for val in data_dfs:
    print(val.Project.unique())

['ABIDE_Initiative']
['ACPI']
['ADHD200']
['ADHD200']
['ADHD200']
['ADHD200']
['ADHD200']
['ADHD200']
['ADHD200']
['ADHD200']
['ADHD200']
['ADHD200']
['ADHD200']
['ADHD200']
['ADHD200']
['ADHD200']
['ADHD200']
['ADHD200']
['ADHD200']
['ADHD200']
['CORR']
['INDI']
['RocklandSample']
['RocklandSample']
['RocklandSample']
['RocklandSample']


In [285]:
pheno_df = pd.concat(data_dfs, axis=0, ignore_index=True).drop_duplicates()
pheno_df.index = range(pheno_df.shape[0])
pheno_df.keys().tolist()

['ADHD Index',
 'ADHD Measure',
 'ADI_RRB_TOTAL_C',
 'ADI_R_ONSET_TOTAL_D',
 'ADI_R_RSRCH_RELIABLE',
 'ADI_R_SOCIAL_TOTAL_A',
 'ADI_R_VERBAL_TOTAL_BV',
 'ADOS_COMM',
 'ADOS_GOTHAM_RRB',
 'ADOS_GOTHAM_SEVERITY',
 'ADOS_GOTHAM_SOCAFFECT',
 'ADOS_GOTHAM_TOTAL',
 'ADOS_MODULE',
 'ADOS_RSRCH_RELIABLE',
 'ADOS_SOCIAL',
 'ADOS_STEREO_BEHAV',
 'ADOS_TOTAL',
 'AGE_AT_MPRAGE',
 'AGE_AT_SCAN',
 'AGE_AT_SCAN_1',
 'AQ_TOTAL',
 'Age',
 'BMI',
 'COMORBIDITY',
 'CURRENT_MED_STATUS',
 'DATASET',
 'DSM_IV_TR',
 'DX',
 'DX_GROUP',
 'Disclaimer',
 'Dominant Hand',
 'Download Group',
 'EYE_STATUS_AT_SCAN',
 'FILE_ID',
 'FIQ',
 'FIQ_TEST_TYPE',
 'Full2 IQ',
 'Full4 IQ',
 'Gender',
 'HANDEDNESS',
 'HANDEDNESS_CATEGORY',
 'HANDEDNESS_SCORES',
 'Handedness',
 'Hyper/Impulsive',
 'ID',
 'IQ Measure',
 'Inattentive',
 'LMP',
 'MEDICATION_NAME',
 'Med Status',
 'OFF_STIMULANTS_AT_SCAN',
 'PIQ',
 'PIQ_TEST_TYPE',
 'PRECEDING_CONDITION',
 'Performance IQ',
 'Project',
 'QC_Anatomical_1',
 'QC_Anatomical_2',
 'QC_Re

In [286]:
id_cols = [val for val in pheno_df.keys().tolist() if val.lower().endswith('id') and 
           all([key not in val.lower() for key in ['site', 'file']])]
id_cols

['ID',
 'SUBID',
 'SUB_ID',
 'ScanDir ID',
 'ScanDirID',
 'Subject ID',
 'participant_id']

In [287]:
pheno_df[id_cols].apply(lambda x: len([val for val in x if isinstance(val, str) or not pd.np.isnan(val)]), axis=1).unique()

array([1])

In [288]:
ids = []
for row in pheno_df[id_cols].iterrows():
    id = [val for val in row[1] if isinstance(val, str) or not pd.np.isnan(val)]
    id = id[0] if id else None
    if id:
        if isinstance(id, float):
            id = '%d' % id
    ids.append(pheno_df.iloc[row[0]]['Project'].lower() + '_' + id)

In [289]:
pheno_df.index = ids

In [305]:
pheno_df

,ADHD Index,ADHD Measure,ADI_RRB_TOTAL_C,ADI_R_ONSET_TOTAL_D,ADI_R_RSRCH_RELIABLE,ADI_R_SOCIAL_TOTAL_A,ADI_R_VERBAL_TOTAL_BV,ADOS_COMM,ADOS_GOTHAM_RRB,ADOS_GOTHAM_SEVERITY,...,WISC_IV_PSI,WISC_IV_SIM_SCALED,WISC_IV_SYM_SCALED,WISC_IV_VCI,WISC_IV_VOCAB_SCALED,WISC_IV_WMI,age,handedness,participant_id,sex
abide_initiative_51456,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
abide_initiative_51457,NaN,NaN,5,3,1,23,17,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
abide_initiative_51458,NaN,NaN,7,4,1,13,18,6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
abide_initiative_51459,NaN,NaN,2,1,1,12,12,4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
abide_initiative_51460,NaN,NaN,6,3,1,21,11,4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
abide_initiative_51461,NaN,NaN,5,3,1,25,15,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
abide_initiative_51462,NaN,NaN,3,3,1,25,18,7,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
abide_initiative_51463,NaN,NaN,4,3,1,24,18,7,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
abide_initiative_51464,NaN,NaN,5,5,1,22,17,5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
abide_initiative_51465,NaN,NaN,9,NaN,1,23,18,5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [303]:
url_df = pd.concat(url_dfs)
url_df

,atLocation,subj_id
abide_initiative_50642,https://s3.amazonaws.com/fcp-indi-new/data/Pro...,abide_initiative_50642
abide_initiative_50646,https://s3.amazonaws.com/fcp-indi-new/data/Pro...,abide_initiative_50646
abide_initiative_50647,https://s3.amazonaws.com/fcp-indi-new/data/Pro...,abide_initiative_50647
abide_initiative_50649,https://s3.amazonaws.com/fcp-indi-new/data/Pro...,abide_initiative_50649
abide_initiative_50653,https://s3.amazonaws.com/fcp-indi-new/data/Pro...,abide_initiative_50653
abide_initiative_50654,https://s3.amazonaws.com/fcp-indi-new/data/Pro...,abide_initiative_50654
abide_initiative_50656,https://s3.amazonaws.com/fcp-indi-new/data/Pro...,abide_initiative_50656
abide_initiative_50659,https://s3.amazonaws.com/fcp-indi-new/data/Pro...,abide_initiative_50659
abide_initiative_50660,https://s3.amazonaws.com/fcp-indi-new/data/Pro...,abide_initiative_50660
abide_initiative_50663,https://s3.amazonaws.com/fcp-indi-new/data/Pro...,abide_initiative_50663


In [308]:
merged_df = pd.merge(pheno_df, url_df, left_index=True, right_index=True)

In [314]:
pheno_df.drop_duplicates().shape, url_df.drop_duplicates().shape

((7098, 136), (5796, 2))

In [5]:
df = pd.read_csv('https://s3.amazonaws.com/fcp-indi-new/data/Projects/ABIDE_Initiative/Phenotypic_V1_0b.csv')

In [9]:
df
sub = df[['SITE_ID', 'SUB_ID', 'DX_GROUP', 'DSM_IV_TR', 'AGE_AT_SCAN', 'SEX', 'HANDEDNESS_CATEGORY', 'FIQ']]

In [19]:
sub

,SITE_ID,SUB_ID,DX_GROUP,DSM_IV_TR,AGE_AT_SCAN,SEX,HANDEDNESS_CATEGORY,FIQ
0,CALTECH,51456,1,4,55.40,1,R,126.0
1,CALTECH,51457,1,4,22.90,1,Ambi,107.0
2,CALTECH,51458,1,1,39.20,1,R,93.0
3,CALTECH,51459,1,1,22.80,1,R,106.0
4,CALTECH,51460,1,1,34.60,2,Ambi,133.0
5,CALTECH,51461,1,4,37.70,1,R,99.0
6,CALTECH,51462,1,1,20.20,2,R,107.0
7,CALTECH,51463,1,1,20.20,2,R,102.0
8,CALTECH,51464,1,1,20.90,1,Ambi,101.0
9,CALTECH,51465,1,1,20.20,1,R,96.0
